In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Gurtej/Drbot")

model = AutoModelForCausalLM.from_pretrained("Gurtej/Drbot")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00


http://www.thetherapist.com/Transcripts.html

In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai
import time
import torch
openai.api_key = "Your Open AI Key"

In [ ]:
def ask_drbot (input_user):

    # encode the new user input, add the eos_token and return a tensor in Pytorch

    user_input = input_user

    #new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # print(new_user_input_ids)

    # append the new user input tokens to the chat history

    bot_input_ids = tokenizer.encode(tokenizer.eos_token + user_input, return_tensors='pt')

    # generated a response while limiting the total chat history to 1000 tokens,

    chat_history_ids = model.generate(

        bot_input_ids, max_length=200,

        pad_token_id=tokenizer.eos_token_id,

        no_repeat_ngram_size=3,

        do_sample=True,

        top_k=100,

        top_p=0.7,

        temperature=0.1

    )
    drbot_answer=tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return drbot_answer

In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    #inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    #print(inp.value)
    if prompt == '':
      response1 = ''
    else:
      if prompt[-1]!='.':
          prompt = prompt + '.'
      response1 = ask_drbot(prompt)
      context.append({'role':'assistant', 'content':f"{response1}"})
    response2 = get_completion_from_messages(context)
    response = response2
    context.append({'role':'assistant', 'content':f"{response2}"})
    if prompt == '':
        panels.append(
            pn.Row('System:', pn.pane.Markdown('Conversation start...', width=600)))
    else:
        panels.append(
            pn.Row('You:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('psychiatrist:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
#context =  [
#{'role':'system', 'content':'You are a virtual and kind psychiatrist who provide the psychotherapy to the user by chating. But do not repeat response that already exist in the context.'}]
context =  [
{'role':'system', 'content':'You are a virtual and kind psychiatrist who provides psychotherapy to the user by chatting. However, please avoid repeating any responses and imporantly keep the response short like a human conversation. You also need to have more short questions for trying understand what happens and user real feeling before judging the context and level of seriousness to make decision on using proper talking Therapy, such as Cognitive Behavioral Therapy, Rational Emotive Behavior Therapy, Dialectical Behavior Therapy, Mindfulness-Based Cognitive Therapy and Schema Therapy. In addition, if you could provide some great famous saying from great people or suggesting musics and movies to the user, then it will be very helpful.'}]

In [ ]:
import panel as pn  # GUI
pn.extension()
panels = [] # collect display
#get_patient_name_iniContext()
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

RateLimitError: ignored